In [1]:
import tensorflow as tf
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]= "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split

def load_data(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*")))
    masks = sorted(glob(os.path.join(dataset_path, "masks/*")))
    
    train_x, test_x= train_test_split(images, test_size=0.2, random_state=42)
    train_y, test_y= train_test_split(masks, test_size=0.2, random_state=42)
    
    return (train_x, train_y), (test_x, test_y)

def read_image(path):
    x= cv2.imread(path, cv2.IMREAD_COLOR)
    x= cv2.resize(x, (256,256))
    x= x/255.0 #normalization stage
    x=x.astype(np.float32)  #Converts image to (256,256,3)
    return x
def read_mask(path):
    x= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x= cv2.resize(x, (256,256)) #Grayscale already puts image in range of zero and 1 so no need for normalization
    x= x.astype(np.float32)   #converts image to (256,256)
    x= np.expand_dims(x, axis=-1)    #converts image from (256,256) to (256,256,1)
    return x
    
def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path= image_path.decode()
        mask_path= mask_path.decode()
            
        x= read_image(image_path)
        y= read_mask(mask_path)
            
        return x,y
        
    image, mask= tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([256,256,3])
    mask.set_shape([256,256,1])
        
    return image, mask
    
def tf_dataset(images, masks, batch=8):
    dataset= tf.data.Dataset.from_tensor_slices((images, masks))
    dataset= dataset.shuffle(buffer_size=5000)
    dataset= dataset.map(preprocess)
    dataset= dataset.batch(batch)
    dataseta= dataset.prefetch(2)
    return dataset
        
if __name__ == "__main__":
    dataset_path="archive/people_segmentation"
    (train_x, train_y), (test_x, test_y)= load_data(dataset_path)
    
    
    print(f"Training: {len(train_x)} - Masks: {len(train_y)}")
    print(f"Training: {len(test_x)} - Masks: {len(test_y)}")
    
    train_dataset = tf_dataset(train_x, train_y, batch=12)
    #for image, mask in train_dataset:
       # print(image.shape, mask.shape)
        
    """
    10 - elements
    batch - 3
    10//3=3
    3*3=9
    3+1
    """
    batch=12
    train_steps=len(train_x)//batch
    if len(train_x) % batch!=0:
        train_steps+=1
        
    print("Train Steps: ", train_steps)

Training: 4542 - Masks: 4542
Training: 1136 - Masks: 1136
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Train Steps:  379
